In [4]:
import zarr
import numpy as np
from dask_image.imread import imread
import dask.array as da
from glob import glob
from napari import Viewer

In [2]:
v = Viewer()

In [2]:
paths = glob("/Users/aaristov/Downloads/20241108-snouty-test2/*/*.tif")
names = [_.split("/")[-2] for _ in paths]
names

['561', '488', '405']

In [3]:
a = da.array(list(map(imread, paths)))
a

dask.array<stack, shape=(3, 303, 1100, 2304), dtype=uint16, chunksize=(1, 1, 1100, 2304), chunktype=numpy.ndarray>

In [5]:
a[:,:,::4,::4].to_zarr("/Users/aaristov/Downloads/20241108-snouty-test2/bin4.zarr")

In [7]:
b = da.from_zarr("/Users/aaristov/Downloads/20241108-snouty-test2/bin4.zarr/")
b

/Users/aaristov/miniconda3/envs/napari/lib/python3.10/site-packages/zarr/creation.py:250: UserWarning: ignoring keyword argument 'read_only'
  warn('ignoring keyword argument %r' % k)


dask.array<from-zarr, shape=(3, 303, 275, 576), dtype=uint16, chunksize=(1, 1, 275, 576), chunktype=numpy.ndarray>

In [8]:
v.add_image(b, channel_axis=0, name=names)

[<Image layer '561' at 0x32c923e20>,
 <Image layer '488' at 0x3257c2800>,
 <Image layer '405' at 0x3232c8a90>]

In [ ]:
for l in v.layers:
    l.shear=[-1.,0,0]
    l.scale=[1.5,.8,1]

In [8]:
root = zarr.open("/Users/aaristov/Downloads/test.zarr", mode="w", shape=(1,100,100), dtype='u2')

In [9]:
root.attrs["shape"] = [100,100]

In [13]:
root.chunks

(1, 100, 100)

In [11]:
root[:] = 100

In [12]:
root.resize((1,100,100))

In [16]:
root.append(np.zeros((1,100,100), dtype='u2'))

(2, 100, 100)

In [7]:
z = zarr.open_group(store="/Users/aaristov/Downloads/test2.zarr")

In [29]:
y = z.create_groups("405", "488", "561")

In [30]:
y

(<zarr.hierarchy.Group '/405'>,
 <zarr.hierarchy.Group '/488'>,
 <zarr.hierarchy.Group '/561'>)

In [10]:
z['405']['0'] = a.compute()

In [15]:
z['488'].create("0", imread("/Users/aaristov/Downloads/20241108-snouty-test2/488/volume_488_t0000.tif"), chunks=(1, 1100, 2304))

TypeError: Group.create() takes 2 positional arguments but 3 were given

In [12]:
z['405']['0']

(38, 138, 288)

In [31]:
z['405'].create("0", shape=(1,100,100))

<zarr.core.Array '/405/0' (1, 100, 100) float64>

In [32]:
z['405']["0"][:] = 100 

In [33]:
z.attrs["channels"] = ["405","488","561"]

In [34]:
zarr.open_group("/Users/aaristov/Downloads/test2.zarr/")

<zarr.hierarchy.Group '/'>

In [ ]:
z['405']['0'].resize((